In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as p
import time

# driver = webdriver.Firefox()
# driver.get('https://quotes.toscrape.com/js')

# wait = WebDriverWait(driver, 3)
# data = []
# quotes = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'quote'))) # OR quotes = driver.find_elements(By.CLASS_NAME, 'quote')
# for quote in quotes:
#     text = quote.find_element(By.CLASS_NAME, 'text').text
#     data.append({'Quotes': text})

# driver.close()
# df = pd.DataFrame(data)
# df



┌──────────────┐     ┌──────────────┐     ┌─────────────┐
│ Data Sources │ ──▶ │   ETL Jobs   │ ──▶ │ AWS RDS/Postgres │
│ (nba_api,    │     │ (Python,     │     └─────────────┘
│  odds_api,   │     │  scheduled)  │             │
│  lineups API)│     └──────────────┘             ▼
└──────────────┘                               ┌───────────┐
                                               │Model Train│
                                               │ & Persist │
                                               └───────────┘
                                                     │
                                                     ▼
                                               ┌───────────┐
                                               │Streamlit  │
                                               │ Dashboard │
                                               └───────────┘


In [2]:

driver = webdriver.Firefox()
driver.get('https://quotes.toscrape.com/js')

time.sleep(3)
page = 0

while page < 2:
    quotes = driver.find_elements(By.CLASS_NAME, 'quote')
    for i in quotes:
        text = i.find_element(By.CLASS_NAME, 'text').text
        author = i.find_element(By.CLASS_NAME, 'author').text
        print(f'{text} by {author}')
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'li.next a')
        next_button.click()
        page += 1
    except:
        break

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” by Albert Einstein
“It is our choices, Harry, that show what we truly are, far more than our abilities.” by J.K. Rowling
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” by Albert Einstein
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” by Jane Austen
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” by Marilyn Monroe
“Try not to become a man of success. Rather become a man of value.” by Albert Einstein
“It is better to be hated for what you are than to be loved for what you are not.” by André Gide
“I have not failed. I've just found 10,000 ways that won't work.” by Thomas A. Edison
“A woman is like a tea bag; you never know how strong it is until it's in hot water.” by Ele

In [3]:
def getJSON():
    url = "https://api.prizepicks.com/projections?league_id=7"
    driver = webdriver.Firefox()
    driver.get(url)

    wait = WebDriverWait(driver, 10)

    wait.until(EC.presence_of_element_located((By.ID, "json-tab")))
    save_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.save")))
    save_button.click()
    time.sleep(2)
    p.press("enter")
    time.sleep(.5)
    p.press("left")
    time.sleep(.5)
    p.press("enter")
    time.sleep(2)

    driver.quit()

In [4]:
import json
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo

directory = "C:\\Users\\alexg\\Downloads\\projections.json"
def loadJSON(directory):
    with open(directory, 'r') as file:
        data = json.load(file)

    player_names = {}
    for elem in data['included']:
        if elem['type'] == 'new_player':
            player_names[elem['id']] = elem['attributes']['name']
    player_projections = []
    for proj in data['data']:
        if proj['type'] == 'projection':
            player_id = proj['relationships']['new_player']['data']['id']
            player_name = player_names.get(player_id)

            line_score = proj['attributes']['line_score']
            stat_type = proj['attributes']['stat_type']

            start_time = proj['attributes']['start_time']
            dt = datetime.fromisoformat(start_time)
            pacific_time = dt.astimezone(ZoneInfo("America/Los_Angeles"))
            month = pacific_time.strftime("%b")
            day = pacific_time.strftime("%d").lstrip('0')
            formatted_date = pacific_time.strftime(f"{month}-{day}-%Y")
            
            player_projections.append((player_name,stat_type, line_score, formatted_date))
    return player_projections

In [5]:
df = loadJSON(directory)
df

[('Max Strus', 'Rebs+Asts', 9, 'May-6-2025'),
 ('Myles Turner', 'Rebounds', 7.5, 'May-6-2025'),
 ('Myles Turner', 'Pts+Rebs', 29.5, 'May-6-2025'),
 ('Myles Turner', 'Blks+Stls', 2.5, 'May-6-2025'),
 ('Myles Turner', 'FG Made', 5, 'May-6-2025'),
 ('Myles Turner', '3-PT Attempted', 5, 'May-6-2025'),
 ('Myles Turner', 'Pts+Rebs', 21.5, 'May-6-2025'),
 ('Myles Turner', '3-PT Made', 1.5, 'May-6-2025'),
 ('Myles Turner', 'Points', 14.5, 'May-6-2025'),
 ('Myles Turner', '3-PT Made', 3.5, 'May-6-2025'),
 ('Myles Turner', 'Points', 17.5, 'May-6-2025'),
 ('Myles Turner', '3-PT Made', 2.5, 'May-6-2025'),
 ('Myles Turner', 'Assists', 1.5, 'May-6-2025'),
 ('Myles Turner', 'FG Attempted', 10.5, 'May-6-2025'),
 ('Myles Turner', 'Points', 21.5, 'May-6-2025'),
 ('Myles Turner', 'Pts+Rebs', 24.5, 'May-6-2025'),
 ('Myles Turner', 'Points', 10.5, 'May-6-2025'),
 ('Myles Turner', 'Points', 19.5, 'May-6-2025'),
 ('Myles Turner', 'Pts+Rebs', 19.5, 'May-6-2025'),
 ('Myles Turner', 'Points', 9.5, 'May-6-2025')

In [3]:
import requests
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = 'cfdd939977d91848aac23db1944a1cda'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
id = '34e829d65b6a0cbb49573338a86772ec'

odds_response = requests.get(
    f" https://api.the-odds-api.com/v4/sports/basketball_nba/events/{id}/odds?apiKey={API_KEY}&regions=us_dfs&markets=player_points&oddsFormat=american",
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')
else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 7
{'id': '34e829d65b6a0cbb49573338a86772ec', 'sport_key': 'basketball_nba', 'sport_title': 'NBA', 'commence_time': '2025-05-09T23:44:04Z', 'home_team': 'Indiana Pacers', 'away_team': 'Cleveland Cavaliers', 'bookmakers': []}
Remaining requests 483
Used requests 17


In [4]:
try:
    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/basketball_nba/events?apiKey={API_KEY}')
    if odds_response.status_code == 200:
        game_ids =  [game['id'] for game in odds_response.json()]
    else:
        print(f"Failed to retrieve data: {odds_response.status_cod}")
        game_ids = []
except requests.RequestException as e:
    print(f"Request failed: {e}")
    game_ids = []
    
    

In [5]:
game_ids

['34e829d65b6a0cbb49573338a86772ec',
 '57675a019ab81bbf046093942697c7e1',
 '87073ecdaedc9200c7468b70625a7ea8',
 'a72d8efa957a31155a3d4aa1dd2a6b13']

In [11]:
def get_odds(id, market_type, API_KEY):
    try:
        response = requests.get(
        f"https://api.the-odds-api.com/v4/sports/basketball_nba/events/{id}/odds?apiKey={API_KEY}&regions=us_dfs&markets={market_type}&oddsFormat=american",
        )
        if response.status_code == 200:
            data = response.json()
            props = []
            for bookmaker in data['bookmakers']:
                for market in bookmaker['markets']:
                    if market['key'] == market_type:
                        for outcome in market['outcomes']:
                            props.append((
                                market['key'],
                                bookmaker['title'],
                                outcome['description'],
                                outcome['name'],
                                outcome['point'],
                                outcome['price'],
                            ))
            return props
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            return []
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return []

get_odds('57675a019ab81bbf046093942697c7e1','player_points', 'cfdd939977d91848aac23db1944a1cda')

[('player_points', 'PrizePicks', 'Jamal Murray', 'Over', 20.5, -137),
 ('player_points', 'PrizePicks', 'Jamal Murray', 'Under', 20.5, -137),
 ('player_points', 'PrizePicks', 'Michael Porter Jr', 'Over', 11.5, -137),
 ('player_points', 'PrizePicks', 'Michael Porter Jr', 'Under', 11.5, -137),
 ('player_points', 'PrizePicks', 'Luguentz Dort', 'Over', 10.0, -137),
 ('player_points', 'PrizePicks', 'Luguentz Dort', 'Under', 10.0, -137),
 ('player_points', 'PrizePicks', 'Aaron Gordon', 'Over', 16.5, -137),
 ('player_points', 'PrizePicks', 'Aaron Gordon', 'Under', 16.5, -137),
 ('player_points', 'PrizePicks', 'Aaron Wiggins', 'Over', 5.5, -137),
 ('player_points', 'PrizePicks', 'Aaron Wiggins', 'Under', 5.5, -137),
 ('player_points', 'PrizePicks', 'Christian Braun', 'Over', 12.5, -137),
 ('player_points', 'PrizePicks', 'Christian Braun', 'Under', 12.5, -137),
 ('player_points', 'PrizePicks', 'Peyton Watson', 'Over', 4.0, -137),
 ('player_points', 'PrizePicks', 'Peyton Watson', 'Under', 4.0, -1

In [1]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()

Scraping Odds API...
Organizing Data...


In [2]:
df = nba_props.dataframe
df
# df.to_csv('./PROPS_DATA/Playoffs(5_9_25)')

,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,PrizePicks,player_points,Myles Turner,Over,15.0,-137
1,PrizePicks,player_points,Myles Turner,Under,15.0,-137
2,PrizePicks,player_points,T.J. McConnell,Over,7.5,-137
3,PrizePicks,player_points,T.J. McConnell,Under,7.5,-137
4,PrizePicks,player_points,Dean Wade,Over,2.5,-137
...,...,...,...,...,...,...
593,PrizePicks,player_turnovers,Draymond Green,Under,2.5,-137
594,Underdog,player_turnovers,Donovan Mitchell,Over,2.5,-137
595,Underdog,player_turnovers,Donovan Mitchell,Under,2.5,-137
596,Underdog,player_blocks_steals,Evan Mobley,Over,2.5,-137


In [3]:
df.to_csv('./PROPS_DATA/Playoffs_PP(5_11_25)')

In [1]:
from nba_api.stats.endpoints import leaguegamelog, boxscoreadvancedv2 

def fetchPlayerStats(season='2022-23'):
    game_logs = leaguegamelog.LeagueGameLog(
        season=season, 
        player_or_team_abbreviation='P'
    ).get_data_frames()[0]
    
    # Process 'MATCHUP' to get 'OPPONENT'
    def extract_opponent(row):
        matchup = row['MATCHUP']
        team_abbr = row['TEAM_ABBREVIATION']
        if ' vs. ' in matchup:
            opponent = matchup.split(' vs. ')[1]
        elif ' @ ' in matchup:
            opponent = matchup.split(' @ ')[1]
        else:
            opponent = None
        return opponent

    game_logs['OPPONENT'] = game_logs.apply(extract_opponent, axis=1)
    game_logs['HOME_GAME'] = game_logs['MATCHUP'].apply(lambda x: 1 if 'vs.' in x else 0)
    game_logs['AWAY_GAME'] = game_logs['MATCHUP'].apply(lambda x: 1 if '@' in x else 0)
    
    # Define column order
    columns = [
        'PLAYER_NAME', 'PLAYER_ID', 'MATCHUP', 'TEAM_ABBREVIATION', 'TEAM_ID', 
        'OPPONENT', 'HOME_GAME','AWAY_GAME', 'GAME_ID', 'GAME_DATE', 'WL', 
        'MIN', 'PTS', 'AST', 'REB', 'FGM', 'FGA', 'FG_PCT', 
        'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
        'OREB', 'DREB', 'STL', 'BLK', 'TOV', 'PF', 
        'PLUS_MINUS', 'FANTASY_PTS'
    ]
    
    # Reorder columns
    game_logs = game_logs[columns]
    print("DATA Complete!")
    
    return game_logs

In [2]:
data = fetchPlayerStats()
data.columns

DATA Complete!


Index(['PLAYER_NAME', 'PLAYER_ID', 'MATCHUP', 'TEAM_ABBREVIATION', 'TEAM_ID',
       'OPPONENT', 'HOME_GAME', 'AWAY_GAME', 'GAME_ID', 'GAME_DATE', 'WL',
       'MIN', 'PTS', 'AST', 'REB', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'STL', 'BLK', 'TOV',
       'PF', 'PLUS_MINUS', 'FANTASY_PTS'],
      dtype='object')

In [4]:
import numpy as np

def eFG(FG,FG3,FGA):
    res = np.where(FGA == 0, 0.0, (FG + 0.5 * FG3) / FGA)
    return res.round(3)

def PointsPerShotAttempt(PTS,FGA,FTA):
    res = np.where(FGA == 0, 0.0,PTS/(FGA+0.44*FTA))
    return res.round(3)


data['eFG'] = eFG(data['FGM'], data['FG3M'], data['FGA'])
data['PPS'] = PointsPerShotAttempt(data['PTS'], data['FGA'], data['FTA'])
data

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPPONENT,HOME_GAME,AWAY_GAME,GAME_ID,GAME_DATE,...,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,eFG,PPS
0,Jordan Poole,1629673,GSW vs. LAL,GSW,1610612744,LAL,1,0,0022200002,2022-10-18,...,0,2,2,0,4,2,1,26.9,0.333,0.756
1,LeBron James,2544,LAL @ GSW,LAL,1610612747,GSW,0,1,0022200002,2022-10-18,...,5,10,0,0,5,2,-10,56.0,0.519,1.117
2,Jonathan Kuminga,1630228,GSW vs. LAL,GSW,1610612744,LAL,1,0,0022200002,2022-10-18,...,1,2,0,1,2,2,-8,4.6,0.000,0.000
3,Juan Toscano-Anderson,1629308,LAL @ GSW,LAL,1610612747,GSW,0,1,0022200002,2022-10-18,...,0,4,0,0,2,1,0,2.8,0.000,0.000
4,Wenyen Gabriel,1629117,LAL @ GSW,LAL,1610612747,GSW,0,1,0022200002,2022-10-18,...,1,0,0,0,0,2,0,2.7,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25890,Kendrick Nunn,1629134,WAS vs. HOU,WAS,1610612764,HOU,1,0,0022201222,2023-04-09,...,1,4,1,2,4,2,-8,27.5,0.556,1.215
25891,Frank Kaminsky,1626163,HOU @ WAS,HOU,1610612745,WAS,0,1,0022201222,2023-04-09,...,2,4,0,1,0,0,4,21.7,0.500,1.000
25892,Johnny Davis,1631098,WAS vs. HOU,WAS,1610612764,HOU,1,0,0022201222,2023-04-09,...,3,5,2,0,2,4,-13,38.6,0.275,0.597
25893,Jalen Green,1630224,HOU @ WAS,HOU,1610612745,WAS,0,1,0022201222,2023-04-09,...,0,2,1,0,0,0,3,25.4,0.500,1.097


In [28]:
data = data.head()
data

PLAYER_NAME          Scotty Pippen Jr.
PLAYER_ID                      1630590
MATCHUP                      LAL @ GSW
TEAM_ABBREVIATION                  LAL
TEAM_ID                     1610612747
Name: 0, dtype: object

In [25]:
df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE'],
      dtype='object')

In [6]:
def fetch_advanced_stats_for_game(game_id, sleep_time=1):
   try:
       time.sleep(sleep_time)  # Sleep to respect rate limits
       boxscore = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
       advanced_stats = boxscore.get_data_frames()[0]  # Adjust index if necessary
       return advanced_stats
   except Exception as e:
       print(f"Error fetching advanced stats for game {game_id}: {e}")
       return pd.DataFrame()
ids = []
for id in data['GAME_ID']:
    if id not in ids:
        ids.append(id)
ids

['0022200002',
 '0022200001',
 '0022200008',
 '0022200007',
 '0022200006',
 '0022200012',
 '0022200011',
 '0022200004',
 '0022200003',
 '0022200005',
 '0022200014',
 '0022200013',
 '0022200009',
 '0022200010',
 '0022200015',
 '0022200016',
 '0022200024',
 '0022200025',
 '0022200023',
 '0022200027',
 '0022200026',
 '0022200019',
 '0022200018',
 '0022200017',
 '0022200022',
 '0022200020',
 '0022200021',
 '0022200034',
 '0022200032',
 '0022200031',
 '0022200033',
 '0022200030',
 '0022200029',
 '0022200028',
 '0022200036',
 '0022200035',
 '0022200043',
 '0022200042',
 '0022200038',
 '0022200041',
 '0022200039',
 '0022200037',
 '0022200040',
 '0022200051',
 '0022200050',
 '0022200049',
 '0022200048',
 '0022200046',
 '0022200045',
 '0022200044',
 '0022200047',
 '0022200055',
 '0022200052',
 '0022200054',
 '0022200053',
 '0022200056',
 '0022200057',
 '0022200059',
 '0022200064',
 '0022200063',
 '0022200058',
 '0022200065',
 '0022200061',
 '0022200062',
 '0022200060',
 '0022200068',
 '00222000

In [7]:
import pandas as pd 
import time
adv = fetch_advanced_stats_for_game(game_id='0022200002')
adv

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022200002,1610612747,LAL,Los Angeles,1629022,Lonnie Walker IV,Lonnie,F,,29.000000:17,...,13.3,0.286,0.317,0.125,0.125,115.82,113.92,94.93,70,0.044
1,0022200002,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,,35.000000:12,...,12.5,0.519,0.558,0.327,0.327,118.06,115.91,96.59,85,0.195
2,0022200002,1610612747,LAL,Los Angeles,203076,Anthony Davis,Anthony,C,,35.000000:36,...,10.3,0.455,0.520,0.296,0.298,120.49,117.98,98.31,87,0.104
3,0022200002,1610612747,LAL,Los Angeles,201566,Russell Westbrook,Russell,G,,30.000000:41,...,19.0,0.625,0.669,0.214,0.217,121.46,118.89,99.08,75,0.161
4,0022200002,1610612747,LAL,Los Angeles,201976,Patrick Beverley,Patrick,G,,24.000000:40,...,12.5,0.300,0.300,0.102,0.104,107.73,108.00,90.00,55,0.015
5,0022200002,1610612747,LAL,Los Angeles,1630559,Austin Reaves,Austin,,,16.000000:00,...,0.0,0.333,0.436,0.068,0.068,125.76,120.00,100.00,40,0.019
6,0022200002,1610612747,LAL,Los Angeles,1629134,Kendrick Nunn,Kendrick,,,23.000000:03,...,21.4,0.722,0.722,0.188,0.193,122.86,119.74,99.78,56,0.085
7,0022200002,1610612747,LAL,Los Angeles,1629308,Juan Toscano-Anderson,Juan,,,14.000000:18,...,40.0,0.000,0.000,0.132,0.138,118.69,115.80,96.50,34,-0.077
8,0022200002,1610612747,LAL,Los Angeles,1629117,Wenyen Gabriel,Wenyen,,,7.000000:48,...,0.0,0.000,0.000,0.095,0.101,109.29,113.85,94.87,18,-0.438
9,0022200002,1610612747,LAL,Los Angeles,1630346,Matt Ryan,Matt,,,17.000000:49,...,20.0,0.375,0.375,0.111,0.110,111.86,111.81,93.17,41,-0.017


In [8]:
adv.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN',
       'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE'],
      dtype='object')

In [14]:
from NBAData.fetchPlayersStats import FetchPlayersStats

# Create the processor
nba_processor = FetchPlayersStats()

# Or get complete stats with advanced metrics
data_25 = nba_processor.getCompleteStats(season='2024-25', sample_size=None, sleep_time=2.5, max_workers=5)
data_25


Fetching player stats...
Basic data completed for 2024-25
Fetching advanced player stats...
Fetched advanced stats for 1 of 1230 games
Fetched advanced stats for 2 of 1230 games
Fetched advanced stats for 3 of 1230 games
Fetched advanced stats for 4 of 1230 games
Fetched advanced stats for 5 of 1230 games
Fetched advanced stats for 6 of 1230 games
Fetched advanced stats for 7 of 1230 games
Fetched advanced stats for 8 of 1230 games
Fetched advanced stats for 9 of 1230 games
Fetched advanced stats for 10 of 1230 games
Fetched advanced stats for 11 of 1230 games
Fetched advanced stats for 12 of 1230 games
Fetched advanced stats for 13 of 1230 games
Fetched advanced stats for 14 of 1230 games
Fetched advanced stats for 15 of 1230 games
Fetched advanced stats for 16 of 1230 games
Fetched advanced stats for 17 of 1230 games
Fetched advanced stats for 18 of 1230 games
Fetched advanced stats for 19 of 1230 games
Fetched advanced stats for 20 of 1230 games
Fetched advanced stats for 21 of 1230

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,AWAY_GAME,GAME_ID,GAME_DATE,...,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,Karl-Anthony Towns,1626157,NYK @ BOS,NYK,1610612752,BOS,0,1,0022400061,2024-10-22,...,119.727592,6.0,3.0,40.0,0.505263,1.610613e+09,119.727592,91.04,90.78,90.52
1,Dalton Knecht,1642261,LAL vs. MIN,LAL,1610612747,MIN,1,0,0022400062,2024-10-22,...,112.244898,4.0,1.0,47.0,0.411765,1.610613e+09,112.244898,98.00,98.94,99.88
2,Jaden McDaniels,1630183,MIN @ LAL,MIN,1610612750,LAL,0,1,0022400062,2024-10-22,...,103.123748,7.0,8.0,46.0,0.442105,1.610613e+09,103.123748,99.88,98.94,98.00
3,Naz Reid,1629675,MIN @ LAL,MIN,1610612750,LAL,0,1,0022400062,2024-10-22,...,103.123748,7.0,8.0,46.0,0.442105,1.610613e+09,103.123748,99.88,98.94,98.00
4,Nickeil Alexander-Walker,1629638,MIN @ LAL,MIN,1610612750,LAL,0,1,0022400062,2024-10-22,...,103.123748,7.0,8.0,46.0,0.442105,1.610613e+09,103.123748,99.88,98.94,98.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26301,Ron Harper Jr.,1631199,DET @ MIL,DET,1610612765,MIL,0,1,0022401192,2025-04-13,...,118.580599,5.0,7.0,38.0,0.581395,1.610613e+09,118.580599,112.16,110.92,109.68
26302,Jamaree Bouyea,1631123,MIL vs. DET,MIL,1610612749,DET,1,0,0022401192,2025-04-13,...,127.644055,9.0,3.0,42.0,0.476636,1.610613e+09,127.644055,109.68,110.92,112.16
26303,Malik Beasley,1627736,DET @ MIL,DET,1610612765,MIL,0,1,0022401192,2025-04-13,...,118.580599,5.0,7.0,38.0,0.581395,1.610613e+09,118.580599,112.16,110.92,109.68
26304,Tolu Smith,1642449,DET @ MIL,DET,1610612765,MIL,0,1,0022401192,2025-04-13,...,118.580599,5.0,7.0,38.0,0.581395,1.610613e+09,118.580599,112.16,110.92,109.68


In [15]:
data_25['OPP_TEAM_ID'] = data_25['OPP_TEAM_ID'].astype(int)
data_25

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,AWAY_GAME,GAME_ID,GAME_DATE,...,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,Karl-Anthony Towns,1626157,NYK @ BOS,NYK,1610612752,BOS,0,1,0022400061,2024-10-22,...,119.727592,6.0,3.0,40.0,0.505263,1610612738,119.727592,91.04,90.78,90.52
1,Dalton Knecht,1642261,LAL vs. MIN,LAL,1610612747,MIN,1,0,0022400062,2024-10-22,...,112.244898,4.0,1.0,47.0,0.411765,1610612750,112.244898,98.00,98.94,99.88
2,Jaden McDaniels,1630183,MIN @ LAL,MIN,1610612750,LAL,0,1,0022400062,2024-10-22,...,103.123748,7.0,8.0,46.0,0.442105,1610612747,103.123748,99.88,98.94,98.00
3,Naz Reid,1629675,MIN @ LAL,MIN,1610612750,LAL,0,1,0022400062,2024-10-22,...,103.123748,7.0,8.0,46.0,0.442105,1610612747,103.123748,99.88,98.94,98.00
4,Nickeil Alexander-Walker,1629638,MIN @ LAL,MIN,1610612750,LAL,0,1,0022400062,2024-10-22,...,103.123748,7.0,8.0,46.0,0.442105,1610612747,103.123748,99.88,98.94,98.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26301,Ron Harper Jr.,1631199,DET @ MIL,DET,1610612765,MIL,0,1,0022401192,2025-04-13,...,118.580599,5.0,7.0,38.0,0.581395,1610612749,118.580599,112.16,110.92,109.68
26302,Jamaree Bouyea,1631123,MIL vs. DET,MIL,1610612749,DET,1,0,0022401192,2025-04-13,...,127.644055,9.0,3.0,42.0,0.476636,1610612765,127.644055,109.68,110.92,112.16
26303,Malik Beasley,1627736,DET @ MIL,DET,1610612765,MIL,0,1,0022401192,2025-04-13,...,118.580599,5.0,7.0,38.0,0.581395,1610612749,118.580599,112.16,110.92,109.68
26304,Tolu Smith,1642449,DET @ MIL,DET,1610612765,MIL,0,1,0022401192,2025-04-13,...,118.580599,5.0,7.0,38.0,0.581395,1610612749,118.580599,112.16,110.92,109.68


In [16]:
import pandas as pd 
data_25.to_csv('./PLAYER_DATA/season_25.csv')
# data_25 = pd.read_csv('PLAYER_DATA/season_25.csv')
# data_25['PointsPerShoot'] = PointsPerShotAttempt(data_25['PTS'],data_25['FGA'],data_25['FTA'])

In [18]:
len(data_25)

26306

In [19]:
import pandas as pd
pd.set_option('display.max_columns', None)
data_25[data_25['PLAYER_NAME'] == 'James Harden']

,Unnamed: 0,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,AWAY_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,PointsPerShoot
117,117,James Harden,201935,LAC vs. PHX,LAC,1610612746,PHX,1,0,22400071,2024-10-23,L,40,29,8,12,10,28,0.357,2,9,0.222,7,9,0.778,3,9,1,0,8,3,-4,50.4,G,100.0,103.4,-3.4,0.068,0.281,0.158,0.381,1.00,0.394,0.454,104.11,105.39,0.126,87.82,0.907
652,652,James Harden,201935,LAC @ DEN,LAC,1610612746,DEN,0,1,22400087,2024-10-26,W,38,23,16,6,5,11,0.455,1,5,0.200,12,13,0.923,0,6,1,1,7,5,3,53.2,G,118.7,110.3,8.4,0.000,0.154,0.088,0.593,2.29,0.296,0.688,97.37,97.27,0.172,81.06,1.376
827,827,James Harden,201935,LAC @ GSW,LAC,1610612746,GSW,0,1,22400101,2024-10-27,W,36,23,11,7,6,19,0.316,2,12,0.167,9,9,1.000,1,6,0,0,5,1,4,42.9,G,111.5,105.1,6.5,0.029,0.154,0.095,0.423,2.20,0.322,0.501,104.70,105.69,0.144,88.08,1.002
1357,1357,James Harden,201935,LAC vs. POR,LAC,1610612746,POR,1,0,22400127,2024-10-30,L,32,19,10,5,6,18,0.333,5,11,0.455,2,4,0.500,0,5,3,0,5,2,1,44.0,G,108.2,104.8,3.4,0.000,0.161,0.070,0.526,2.00,0.333,0.481,96.28,93.07,0.139,77.56,0.962
1518,1518,James Harden,201935,LAC vs. PHX,LAC,1610612746,PHX,1,0,22400131,2024-10-31,L,36,25,13,10,8,19,0.421,3,10,0.300,6,7,0.857,0,10,2,2,6,2,-2,62.5,G,121.3,122.4,-1.0,0.000,0.357,0.145,0.481,2.17,0.311,0.566,99.52,101.56,0.170,84.63,1.132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24811,24811,James Harden,201935,LAC vs. DAL,LAC,1610612746,DAL,1,0,22401132,2025-04-05,W,31,29,14,7,10,19,0.526,1,4,0.250,8,9,0.889,2,5,1,0,2,2,26,59.4,G,145.5,106.1,39.4,0.077,0.152,0.119,0.483,7.00,0.324,0.632,103.21,101.21,0.232,84.35,1.263
25316,25316,James Harden,201935,LAC vs. SAS,LAC,1610612746,SAS,1,0,22401155,2025-04-08,W,38,21,12,9,8,17,0.471,3,6,0.500,2,3,0.667,1,8,0,0,3,2,14,46.8,G,131.6,111.7,19.9,0.025,0.235,0.122,0.375,4.00,0.223,0.573,100.43,97.13,0.158,80.94,1.146
25405,25405,James Harden,201935,LAC vs. HOU,LAC,1610612746,HOU,1,0,22401166,2025-04-09,W,37,35,10,3,10,18,0.556,5,11,0.455,10,10,1.000,1,2,2,0,2,1,11,57.6,G,134.6,122.1,12.5,0.034,0.065,0.050,0.333,5.00,0.276,0.781,100.08,100.13,0.184,83.45,1.562
25784,25784,James Harden,201935,LAC @ SAC,LAC,1610612746,SAC,0,1,22401184,2025-04-11,W,43,23,10,11,8,15,0.533,2,6,0.333,5,7,0.714,2,9,2,1,5,2,3,55.2,G,113.6,107.2,6.4,0.059,0.200,0.139,0.357,2.00,0.261,0.636,91.81,92.39,0.188,77.00,1.272
